<a href="https://colab.research.google.com/github/mainek2004/DataAnalysis_25_26/blob/khoa/EDA_Ph%C3%A2n_lo%E1%BA%A1i_Ti%E1%BB%83u_%C4%91%C6%B0%E1%BB%9Dng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# EDA Phân loại Tiểu đường (Dữ liệu mô phỏng)

Notebook này minh họa quy trình EDA và phân loại đơn giản dựa trên các ngưỡng xấp xỉ tiêu chuẩn **NDDG 1979** (mục đích học tập, không dùng để chẩn đoán y khoa).

**Tiêu chí chẩn đoán (giản lược để minh họa):**
- **Diabetes** nếu `FPG ≥ 140 mg/dL` **hoặc** `OGTT 2h ≥ 200 mg/dL`  
- **IGT** nếu `FPG < 140 mg/dL` **và** `OGTT 2h ∈ [140, 199]`  
- **Normal** nếu không rơi vào hai nhóm trên  
- **GDM** (tiểu đường thai kỳ, minh họa): Nữ đang mang thai và `OGTT 2h ≥ 165 mg/dL` (proxy đơn giản)

> Bạn có thể thay dữ liệu mô phỏng bằng file thực (CSV/XLSX) ở phần 6.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



## 1) Cài đặt & thư viện
- Dùng `matplotlib` cho biểu đồ (không seaborn)
- Đặt seed để tái lập kết quả


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(42)



## 2) Tạo dữ liệu mô phỏng
Sinh ngẫu nhiên các đặc trưng: tuổi, giới tính, BMI, huyết áp, tiền sử gia đình, marker HLA, tình trạng mang thai, glucose lúc đói (FPG) và OGTT 2h.


In [ ]:

N = 600

# Nhân khẩu học (tuổi, giới tính)
age = np.random.randint(18, 80, size=N)
sex = np.random.choice(['F', 'M'], size=N, p=[0.52, 0.48])

# Phân bố BMI với phần đuôi nặng hơn (nhiều giá trị lớn)
bmi = np.clip(np.random.normal(26, 5, size=N), 16, 50)

# Huyết áp tâm thu (ước lượng gần đúng)
bp_sys = np.clip(np.random.normal(122, 15, size=N) + 0.8*(bmi-25), 90, 200)

# Tiền sử gia đình (người thân bậc 1)
family_hist = np.random.choice([0,1], size=N, p=[0.7, 0.3])

# Dấu hiệu HLA (nguy cơ mắc tiểu đường type 1)
hla_risk = np.random.choice([0,1], size=N, p=[0.88, 0.12])

# Tình trạng mang thai (chỉ áp dụng cho nữ trong độ tuổi sinh sản 18–45)
pregnancy = np.zeros(N, dtype=int)
female_idx = np.where((sex=='F') & (age>=18) & (age<=45))[0]
pregnancy[female_idx] = np.random.choice([0,1], size=len(female_idx), p=[0.9, 0.1])

# Sinh giá trị glucose lúc đói và OGTT 2h có điều kiện theo BMI, tiền sử gia đình
base_fpg = np.random.normal(95, 15, size=N)
base_2h = np.random.normal(120, 30, size=N)

# Tăng thêm theo BMI và tiền sử gia đình; thêm nhóm nhỏ giống type 1 có HLA risk -> glucose cao
fpg = base_fpg + 0.8*(bmi-25) + 8*family_hist + 18*(hla_risk*np.random.binomial(1, 0.25, size=N))
ogtt_2h = base_2h + 1.2*(bmi-25) + 12*family_hist + 25*(hla_risk*np.random.binomial(1, 0.25, size=N))

# Giới hạn trong khoảng thực tế
fpg = np.clip(fpg, 60, 350)
ogtt_2h = np.clip(ogtt_2h, 70, 450)

# HbA1c ánh xạ thô từ glucose + thêm nhiễu
hba1c = np.clip(4.5 + (fpg-90)/30 + np.random.normal(0, 0.4, size=N), 4.5, 14.0)

df = pd.DataFrame({
    'age': age,
    'sex': sex,
    'bmi': bmi.round(2),
    'bp_sys': bp_sys.round(0),
    'family_history': family_hist,
    'hla_risk_marker': hla_risk,
    'pregnancy': pregnancy,
    'fasting_glucose_mgdl': fpg.round(1),
    'ogtt_2h_mgdl': ogtt_2h.round(1),
    'hba1c_percent': hba1c.round(2),
})
df.head()


,age,sex,bmi,bp_sys,family_history,hla_risk_marker,pregnancy,fasting_glucose_mgdl,ogtt_2h_mgdl,hba1c_percent
0,21,M,22.33,149.0,0,0,0,75.1,169.2,4.50
1,46,F,39.29,141.0,0,0,0,116.1,78.0,5.56
2,74,M,35.00,107.0,0,0,0,99.4,115.6,5.17
3,18,F,25.65,146.0,0,0,0,88.0,139.4,4.50
4,60,F,33.67,118.0,1,0,0,114.2,177.7,5.33



## 3) Quy tắc chẩn đoán (giản lược)
Áp dụng rule-based theo các ngưỡng đã nêu.


In [ ]:

# Quy tắc chẩn đoán
diagnosis = np.where(
    (df['fasting_glucose_mgdl'] >= 140) | (df['ogtt_2h_mgdl'] >= 200), 'Diabetes',   # Nếu FPG ≥ 140 hoặc OGTT ≥ 200 → Tiểu đường
    np.where(
        (df['fasting_glucose_mgdl'] < 140) & (df['ogtt_2h_mgdl'] >= 140) & (df['ogtt_2h_mgdl'] < 200),
        'IGT',  # Nếu FPG < 140 và OGTT nằm trong khoảng [140, 200) → Rối loạn dung nạp glucose (IGT)
        'Normal'  # Các trường hợp còn lại → Bình thường
    )
)

# Dấu hiệu tiểu đường thai kỳ (GDM): đối với nữ đang mang thai, nếu OGTT 2h ≥ 165 thì gán nhãn 'GDM'
gdm_mask = (df['sex']=='F') & (df['pregnancy']==1) & (df['ogtt_2h_mgdl'] >= 165)
diagnosis = np.where(gdm_mask, 'GDM', diagnosis)

# Thêm cột chẩn đoán vào DataFrame
df['diagnosis'] = diagnosis

# Thêm cột béo phì: BMI ≥ 30 được coi là béo phì (1 = béo phì, 0 = không)
df['obese'] = (df['bmi'] >= 30).astype(int)

# Đếm số lượng từng loại chẩn đoán
df['diagnosis'].value_counts()


,count
diagnosis,
Normal,401
IGT,188
Diabetes,10
GDM,1



## 4) Thống kê mô tả
Tạo các bảng tổng hợp theo chẩn đoán và theo trạng thái béo phì.


In [ ]:

# Tạo bảng tổng hợp: đếm số lượng theo từng loại chẩn đoán
summary_counts = df['diagnosis'].value_counts().rename_axis('diagnosis').reset_index(name='count')

# Tạo bảng tổng hợp: đếm số lượng theo từng loại chẩn đoán và tình trạng béo phì (obese)
summary_by_obesity = df.groupby(['diagnosis','obese']).size().reset_index(name='count')

# Tạo bảng thống kê: tính trung bình các chỉ số (FPG, OGTT, HbA1c, BMI, huyết áp, tuổi) theo từng nhóm chẩn đoán
summary_stats = (
    df.groupby('diagnosis')[['fasting_glucose_mgdl','ogtt_2h_mgdl','hba1c_percent','bmi','bp_sys','age']]
      .mean()
      .round(2)
      .reset_index()
)

# Hiển thị các bảng kết quả
display(summary_counts)
display(summary_by_obesity)
display(summary_stats)



## 5) Trực quan hóa
Mỗi biểu đồ nằm trên một figure, không set màu thủ công.


In [ ]:

import matplotlib.pyplot as plt

# Vẽ biểu đồ histogram cho chỉ số glucose lúc đói
plt.figure()
df['fasting_glucose_mgdl'].hist(bins=30)   # tạo biểu đồ tần suất với 30 cột (bins)
plt.title("Phân bố Glucose lúc đói (mg/dL)")  # tiêu đề biểu đồ
plt.xlabel("FPG (mg/dL)")   # nhãn trục X
plt.ylabel("Số lượng")      # nhãn trục Y
plt.show()  # hiển thị biểu đồ


In [ ]:
# Vẽ biểu đồ histogram cho chỉ số glucose sau 2 giờ làm nghiệm pháp dung nạp glucose (OGTT)
plt.figure()
df['ogtt_2h_mgdl'].hist(bins=30)   # tạo biểu đồ tần suất với 30 cột (bins)
plt.title("Phân bố Glucose sau 2h OGTT (mg/dL)")  # tiêu đề biểu đồ
plt.xlabel("2h OGTT (mg/dL)")  # nhãn trục X
plt.ylabel("Số lượng")         # nhãn trục Y
plt.show()  # hiển thị biểu đồ

In [ ]:

# Vẽ biểu đồ cột (bar chart) cho số lượng bệnh nhân theo từng nhóm chẩn đoán
plt.figure()
df['diagnosis'].value_counts().plot(kind='bar')  # vẽ cột dựa trên số lượng của từng loại chẩn đoán
plt.title("Số lượng theo chẩn đoán")  # tiêu đề biểu đồ
plt.xlabel("Chẩn đoán")              # nhãn trục X (các nhóm chẩn đoán: Normal, IGT, Diabetes, GDM)
plt.ylabel("Số lượng")               # nhãn trục Y (số người trong mỗi nhóm)
plt.show()  # hiển thị biểu đồ

In [ ]:

# Vẽ biểu đồ phân tán (scatter plot) giữa BMI và chỉ số OGTT sau 2h
plt.figure()
plt.scatter(df['bmi'], df['ogtt_2h_mgdl'], s=10, alpha=0.6)
# s=10: kích thước điểm, alpha=0.6: độ trong suốt để dễ thấy chồng điểm

plt.title("BMI vs OGTT 2h")      # tiêu đề biểu đồ
plt.xlabel("BMI")                # nhãn trục X: chỉ số khối cơ thể
plt.ylabel("2h OGTT (mg/dL)")    # nhãn trục Y: glucose sau 2h làm OGTT
plt.show()  # hiển thị biểu đồ


## 6) (Tuỳ chọn) Dùng dữ liệu thật của bạn
Thay vì dữ liệu mô phỏng, bạn có thể đọc file CSV/XLSX của mình rồi **chạy lại** các ô chẩn đoán & biểu đồ.

```python
# Ví dụ đọc CSV của bạn:
# df = pd.read_csv('/path/to/your_file.csv')

# Cần có các cột: 'fasting_glucose_mgdl', 'ogtt_2h_mgdl', 'sex', 'age', 'bmi'
# (tuỳ chọn) 'pregnancy' với 1/0 nếu muốn bật logic GDM.
```

Sau khi nạp df, chạy lại các ô ở mục 3 → 5.
